# Mapping 1D representation of a Coulomb Matrix to XYZ coordinates using KD Tree algorithm

In [1]:
from generatexyz import Generatexyz
import numpy as np
from gan import load_weight
from gan import GenAdvNetwork
import tensorflow as tf
from utils import random_generator
from reformatToXyz import format_xyz_samples
import os
import pathlib
from reformatToXyz import MapCoulomb1dToXYZ
from formatData import loadData
from sklearn.model_selection import train_test_split
from coulombToTraj import getFlattenedXyz
import json

2023-05-07 15:02:49.320271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Generating the Sample Space

In [2]:
# benzene_xyz1 = [-0.17145689,  1.62329681, -0.14969309,  1.0532756,   0.74271863, -0.21060008,
#                   1.25035631, -0.82542328,  0.18999058,  0.14934203, -1.41046102,  0.07804137,
#                  -1.1217173,  -0.94694526,  0.05950565, -1.18889527,  0.65853499, -0.04727168,
#                  -0.15371834,  2.62676478,  0.08424257,  2.30316207,  1.16935347,  0.112503,
#                   2.1246294,  -1.19516863,  0.10442406,  0.01927036, -2.48296332, -0.05464662,
#                  -2.17524543, -1.20036269, -0.07587276, -2.3865359,   1.29179936,  0.07267016]

# # benzene_xyz1 = [-7.709999999999999840e-04,1.397329999999999961e+00,-4.369999999999999970e-04,
# #                1.209629999999999983e+00,6.984810000000000185e-01,3.699999999999999947e-04,
# #                1.209920000000000107e+00,-6.979300000000000503e-01,-2.930000000000000204e-04,
# #                -1.159000000000000091e-03,-1.396020000000000039e+00,-9.120000000000000502e-04,
# #                -1.209799999999999986e+00,-6.975620000000000154e-01,-1.179999999999999951e-04,
# #                -1.209910000000000041e+00,6.974920000000000009e-01,7.039999999999999817e-04,
# #                4.121999999999999796e-03,2.482200000000000184e+00,2.606999999999999942e-03,
# #                2.150770000000000071e+00,1.239430000000000032e+00,2.688999999999999984e-03,
# #                2.150990000000000180e+00,-1.241689999999999960e+00,-5.223000000000000212e-03,
# #                -1.544999999999999932e-03,-2.478429999999999911e+00,-5.577000000000000360e-03,
# #                -2.147580000000000044e+00,-1.239800000000000013e+00,-3.677000000000000147e-03,
# #                -2.152169999999999916e+00,1.239570000000000061e+00,-1.472000000000000001e-03]
# benzene_xyz1 = np.array(benzene_xyz1)
# gen_obj = Generatexyz(500000, 1000, 12)
# gen_obj.generate_samples(benzene_xyz1)

# with open(os.path.abspath(os.path.dirname('__file__')) + "/data/exp/cacheData.json", 'r') as openfile:
#     json_obj = json.load(openfile)
# json_obj["currentBatch"] = 0
# with open(os.path.abspath(os.path.dirname('__file__')) + "/data/exp/cacheData.json", 'w') as openfile:
#     json.dump(json_obj, openfile)  

# benzene_xyz2 = [129.57232951, 128.37783675, 130.41706718, 
#                 129.15831316, 128.05932579, 129.1571882,  
#                 128.70587219, 129.06570857, 128.19918477, 
#                 129.14619152, 130.3506034,  128.37733883, 
#                 129.77042979, 130.67035691, 129.52863719,
#                 129.89559146, 129.7180876,  130.54515808, 
#                 129.63946212, 127.78174293, 131.29582627, 
#                 128.76698797, 127.05313918, 128.88230508, 
#                 128.38464609, 128.67412533, 127.18115179, 
#                 129.11318419, 130.98525282, 127.60574842, 
#                 130.08501537, 131.67039621, 129.71341141, 
#                 130.46258637, 130.1117458, 131.38306266]
# benzene_xyz2 = np.array(benzene_xyz2)
# gen_obj.generate_samples(benzene_xyz2)

# # with open(os.path.abspath(os.path.dirname('__file__')) + "/data/exp/cacheData.json", 'r') as openfile:
# #     json_obj = json.load(openfile)
# # json_obj["currentBatch"] = 0
# # with open(os.path.abspath(os.path.dirname('__file__')) + "/data/exp/cacheData.json", 'w') as openfile:
# #     json.dump(json_obj, openfile) 

# # benzene_xyz3 = [113.830002,115.110008,113.190002,
# #                 112.890007,114.090004,113.150009,
# #                 113.090012,112.949997,113.930008,
# #                 114.170006,112.850006,114.770004,
# #                 115.070007,113.890007,114.880005,
# #                 114.910004,115.040009,114.059998,
# #                 113.690002,115.950005,112.520004,
# #                 112.010002,114.220009,112.529999,
# #                 112.360001,112.150002,113.930008,
# #                 114.380005,111.959999,115.350006,
# #                 115.880005,113.919998,115.610008,
# #                 115.580002,115.890007,114.170006]                
# # benzene_xyz3 = np.array(benzene_xyz3)
# # gen_obj.generate_samples(benzene_xyz3)
# gen_obj.sorting_by_coulomb_matrix()

### Using GAN to generate 1D Coulomb Matrix Representation

In [3]:
latent_dim_ = 32
batch_size_ = 16
weights_path = "/home/panthibivek/thesis/GAN_pkg/runs/train/exp2/weights/"
GanModel = GenAdvNetwork(latent_dim=latent_dim_, batch_size=batch_size_)
GanModel.compile(
    generator_opt=tf.keras.optimizers.Adam(learning_rate=0.001),
    discriminator_opt=tf.keras.optimizers.Adam(learning_rate=0.001),
    disc_loss=tf.keras.losses.BinaryCrossentropy(),
    gen_loss=tf.keras.losses.MAE
)

random_data = random_generator((100, latent_dim_))

GanModel.generator(random_data)
GanModel.load_weights(weights_path) 
generated_output = GanModel.generator(random_data)
arr = generated_output.numpy()
arr_new = arr.reshape((100,78))

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                429       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 13)                0         
                                                                 
 reshape (Reshape)           (None, 13, 1)             0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 39, 128)          640       
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 39, 128)           0         
                                                                 
 batch_normalization (BatchN  (None, 39, 128)          512       
 ormalization)                                           

2023-05-07 15:02:50.641957: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-07 15:02:50.642039: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-07 15:02:50.698341: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-07 15:02:50.698423: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-07 15:02:50.698473: I tensorflow/compiler/xla/stream_executo

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 26, 64)            320       
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 26, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 13, 264)           67848     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 13, 264)           0         
                                                                 
 batch_normalization_2 (Batc  (None, 13, 264)          1056      
 hNormalization)                                                 
                                                                 
 conv1d_3 (Conv1D)           (None, 7, 128)            135296    
                                                     

2023-05-07 15:02:50.918684: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.919404: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.950631: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.951276: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.975083: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.975893: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.990541: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.991574: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50.992977: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-05-07 15:02:50

### Using KDTree to find the closest xyz coordiants in the Sample Space
#### Time Complexity:
For generating KDtree : O(N log(N)) <br>
For finding each sample : O(log(N)) <br> <br>
Note: The defination of class MapCoulomb1dToXYZ is in reformatTOXyz.py

In [5]:
mapping_obj = MapCoulomb1dToXYZ(abs_path_xyz_dirname=str("/home/panthibivek/thesis/GAN_pkg/data/newMappedMolecules"), 
                                sorted_xyz_file="/home/panthibivek/thesis/GAN_pkg/data/exp/totalSortedTraj.txt", 
                                sorted_coulomb1D_file="/home/panthibivek/thesis/GAN_pkg/data/exp/totalSortedCoulomb1D.txt")

In [8]:
get_xyz_small_batch = mapping_obj.generateXYZ(coulomb1D_arr=arr_new)